## OEFENZITTING 4: UITBREIDINGSVELDEN EN EINDIGE VELDEN

In [1]:
from FiniteField import *
from util import *

### 1. Splits in irreduceerbare factoren over $\mathbb{Z}_3$
    1. x^5 + 2x^4 + x^3 + x^2 + 2
    2. x^7 + x^6 + x^5 - x^3 + x^2 - x - 1

In [2]:
Z3 = IntegerField(3)
x = Z3.x()
p = Polynomial(Z3[2,0,1,1,2,1])
p

2 + x² + x³ + 2x⁴ + x⁵

eerst de nulpunten

In [3]:
p.find_roots()

[2]

In [4]:
factors = [x - 2]
p = p / factors[0]
p

2 + x + x³ + x⁴

Deze heeft geen nulpunten meer, maar zou nog wel gereduceerd kunnen worden in irreduceerbare factoren van graad 2.

In [5]:
# TODO: dit is niet efficient. Er zijn truukjes om het beter te doen
# maar ze zijn allemaal nogal brute force.
irrb_graad2 = [f for f in Z3.all_monic_polynomials(2) if f.find_roots() == []]
irrb_graad2

[1 + x², 2 + x + x², 2 + 2x + x²]

In [6]:
for f in irrb_graad2:
    if (p % f).is_zero():
        factors.append(f)
        factors.append(p / f)
        break
factors

[1 + x, 1 + x², 2 + x + x²]

Dit is het antwoord op de eerste deelvraag. Nu de tweede:

In [7]:
p = Polynomial(Z3[-1,-1,1,-1,0,1,1,1])
factors = []
p

2 + 2x + x² + 2x³ + x⁵ + x⁶ + x⁷

In [8]:
p.find_roots()

[]

Geen nulpunten. We kijken of er irreduceerbare factoren zijn van graad 2.

In [9]:
for f in irrb_graad2:
    if (p % f).is_zero():
        factors.append(f)
        p = p / f
print("factoren:", factors)
print("overblijvend:", p)

factoren: [1 + x², 2 + 2x + x²]
overblijvend: 1 + 2x² + x³


Deze overblijvende is irreduceerbaar, want als die wel reduceerbaar was, dan had die nog een nulpunt gehad. Dat betekent dat de oorspronkelijke `p` ook dat nulpunt zou hebben, maar `p` had geen nulpunten. We zijn dus klaar.

### 2. Toon aan dat...
onmogelijk in Python te doen

### 3. Zij $GF(4) = \{0,1,\xi, \xi + 1\}$ met $\xi^2 + \xi + 1 = 0$.

In [10]:
Z2 = IntegerField(2)
GF4 = ExtendedField(Z2, 2,"ξ", Polynomial(Z2[1,1,1]))
GF4

[0, 1, ξ, 1 + ξ]

#### Bepaal alle monische irreduceerbare veeltermen van graad twee over GF(4)

In [11]:
# opnieuw weer niet efficient
irrb_graad2 = [f for f in GF4.all_monic_polynomials(2) if f.find_roots() == []]
irrb_graad2

[1 + ξx + x²,
 1 + (1 + ξ)x + x²,
 ξ + x + x²,
 ξ + ξx + x²,
 1 + ξ + x + x²,
 1 + ξ + (1 + ξ)x + x²]

#### construeer GF(4^2) uit GF(4) m.b.v. de veelterm $x^2 + xξ + ξ$

In [12]:
w = Polynomial(GF4["ξ", "ξ", 1])
w

ξ + ξx + x²

In [13]:
# de computer kan enkel rekenen met veeltermen, niet met artificieel ingevoerde nulpunten,
# maar dat maakt niet uit, want dat is isomorf.
alpha = GF4.x()

machten_alpha = [None for i in range(4**w.degree())]
machten_alpha[0] = GF4[1]
for i in range(1, 4 ** w.degree()):
    machten_alpha[i] = (alpha * machten_alpha[i-1]) % w
    
[str(macht).replace("x", "α") for macht in machten_alpha]

['1',
 'α',
 'ξ + ξα',
 '1 + ξ + α',
 'ξ + α',
 'ξ',
 'ξα',
 '1 + ξ + (1 + ξ)α',
 '1 + ξα',
 '1 + ξ + ξα',
 '1 + ξ',
 '(1 + ξ)α',
 '1 + α',
 'ξ + (1 + ξ)α',
 '1 + (1 + ξ)α',
 '1']

In [14]:
GF16 = ExtendedField(GF4, 2, "α", w)

#### Bepaal de minimaalveeltermen van de elementen van GF(4^2) over GF(4) .

De minimaalveelterm van $\alpha$ is degene die we gekregen hebben (namelijk `w(x)`). De andere vinden we met de cyclotomische nevenklassen.

In [15]:
nevenkl = cyclotomic_cosets(4, 15)
nevenkl

[[0], [1, 4], [2, 8], [3, 12], [5], [6, 9], [7, 13], [10], [11, 14]]

In [16]:
x = GF16.x()
alpha = GF16["α"]

minimaalveeltermen = [
    product([x - alpha**j for j in klasse])
    for klasse in nevenkl
]
minimaalveeltermen

[1 + x,
 ξ + ξx + x²,
 1 + ξ + (1 + ξ)x + x²,
 1 + ξx + x²,
 ξ + x,
 1 + (1 + ξ)x + x²,
 ξ + x + x²,
 1 + ξ + x,
 1 + ξ + x + x²]

#### Bepaal de primitieve veeltermen van graad 2 over GF(4) .

In [17]:
# de computer kan enkel rekenen met veeltermen, niet met artificieel ingevoerde nulpunten,
# maar dat maakt niet uit, want dat is isomorf.
nulpt = GF4.x()

for veelterm in irrb_graad2:
    nulpt_tot_de_i = GF4[1]
    machten = [nulpt_tot_de_i]
    for i in range(1, 4 ** 2 - 1):
        nulpt_tot_de_i = (nulpt * nulpt_tot_de_i) % veelterm
        if nulpt_tot_de_i in machten:
            # deze veelterm is niet primitief
            break
        else:
            machten.append(nulpt_tot_de_i)
    if len(machten) == 4**2 - 1:
        print(veelterm)

ξ + x + x²
ξ + ξx + x²
1 + ξ + x + x²
1 + ξ + (1 + ξ)x + x²


### 4. Zij $GF(9) = \{0,1,2,α,α+ 1,α+ 2,2α,2α+ 1,2α+ 2\}$, met $α^2= 1−α$

In [18]:
GF3 = IntegerField(3)
w = Polynomial(Z3[-1,1,1])
GF9 = ExtendedField(GF3, 2, "α", w)
w

2 + x + x²

#### Ga na dat $x^2+αx+ 1$ irreduceerbaar is over GF(9) .

In [19]:
p = Polynomial(GF9[1, "α", 1])
# heeft p nulpunten?
p.find_roots()

[]

#### Construeer GF(81) door GF(9) uit te breiden met een nulpunt ξ van deze veelterm.

!! Opgelet. Dit is geen primitieve veelterm. Precies omdat de multiplicatieve orde niet 1 is.

In [20]:
xi = GF9.x()
xi_tot_de_i = GF9[1]
for i in range(1, 9 ** p.degree()):
    xi_tot_de_i = (xi * xi_tot_de_i) % p
    if(xi_tot_de_i.is_one()):
        print("De orde van ξ is", i)
        break

De orde van ξ is 10


We zien dat dit een deler is van de grootte van de orde van de multiplicatieve groep, maar deze groep is niet $GF(81)$ omdat we het nulelement moeten uitsluiten. Dus de orde van ξ is inderdaad een deler van 81 - 1 = 80.